In [ ]:
import torch
import torch.nn as nn

batch_size = 4


class SampleConcatener(nn.Module):
    def __init__(self, input_X_dim, input_t_dim, input_s_dim, output_dim, hidden=16):
        super(SampleConcatener, self).__init__()
        self.mlp_embedder = nn.Linear(input_X_dim, hidden)
        self.temporal_embedder = nn.Linear(input_t_dim, lat*lon)
        self.mlp_decoder = nn.Linear(hidden + 1 + input_s_dim, output_dim)

    def forward(self, X, t, s):
        X = self.mlp_embedder(X).relu()
        print(X.shape)
        # (batch, s, temp_features) -> (batch, 1, s*temp_features) -> (batch, 1, lat*lon) -> (batch, lat*lon, 1)
        print(t.shape)
        t = self.temporal_embedder(t.reshape(batch_size, 1, -1)).relu().permute((0,2,1))
        print(t.shape)
        print(s.shape)
        concat = torch.cat((X, t, s), dim=-1)
        print(concat.shape)
        
        print('----------')
        print('GNN cells ...')                
        output = self.mlp_decoder(concat)
        print('----------')
        print(output.shape)
        return output

lat = 32
lon = 48    
s = 3
features = 6
temp_features = 4
spat_features = 6

X = torch.randn(batch_size, lat*lon, s*features)
temp_enc = torch.randn(batch_size, s, temp_features)
spat_enc = torch.randn(batch_size, lat*lon, spat_features)

model = SampleConcatener(input_X_dim=s*features, input_t_dim=s*temp_features, input_s_dim=spat_features, output_dim=features)

output = model(X, temp_enc, spat_enc)


In [ ]:
import sys
sys.path.append('..')
from baselines.cnn.cnn import UNet
import torch

batch_size = 4
lat = 32
lon = 48    
s = 3
fh = 3
features = 6
temp_features = 4
spat_features = 6

X = torch.randn(batch_size, lat*lon, s*features)
temp_enc = torch.randn(batch_size, s, temp_features)
spat_enc = torch.randn(batch_size, lat*lon, spat_features)

model = UNet(features, spat_features, temp_features, features, lat, lon, s, fh)

output = model(X, temp_enc, spat_enc)

In [ ]:
import sys
sys.path.append('..')
from baselines.gnn.cgc_conv import CrystalGNN
import torch

batch_size = 4
lat = 32
lon = 48    
s = 3
fh = 3
features = 6
temp_features = 4
spat_features = 6

X = torch.randn(batch_size, lat*lon, s*features)
temp_enc = torch.randn(batch_size, s, temp_features)
spat_enc = torch.randn(batch_size, lat*lon, spat_features)
edge_index = torch.randn(2, 70544)
edge_attr = torch.randn(70544, 3)

model = CrystalGNN(features, features, 3, 64, temp_features, spat_features)

output = model(X, edge_index, edge_attr, temp_enc, spat_enc)

In [1]:
import sys
sys.path.append('..')
from baselines.gnn.processor import NNDataProcessor

nn_proc = NNDataProcessor(additional_encodings=True)

c:\Users\iwama\Desktop\Maciej\Eng\meteoapp-data\venv\lib\site-packages\gribapi\__init__.py:23: UserWarning: ecCodes 2.31.0 or higher is recommended. You are running version 2.27.0
  warnings.warn(


Loaded data: (4384, 32, 48, 6)
Loaded spatial data: (32, 48, 6)
Loaded temporal data: (4384, 32, 48, 4)


In [2]:
nn_proc.preprocess(subset=None)

AUTOREG_SEQ: (6, 32, 48, 6) (4384, 32, 48, 6)
AUTOREG_SEQ_TP (32, 48, 4) (4384, 32, 48, 4)
AUTOREG_SEQ_SPT (32, 48, 6) (32, 48, 6)
LOADERS: ((4379, 1536, 6, 5), (4379, 1536, 6, 1), (4384, 32, 48, 4), (32, 48, 6))


In [3]:
next(iter(nn_proc.train_loader))

DataBatch(x=[6144, 6, 5], edge_index=[2, 70544], edge_attr=[70544, 3], y=[6144, 6, 1], pos=[128, 48, 6], time=[128, 48, 4], batch=[6144], ptr=[5])